In [6]:
import ipyvuetify as v

In [7]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from bqplot import pyplot as plt, LinearScale, Axis, Bars, OrdinalScale , Label
import requests
import bqplot
from datetime import datetime


# First histogram plot

## Top 10 warehouses with the most racks


In [8]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_profits = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options_profits = [{'text': str(id), 'value': id} for id in warehouse_ids_profits]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_3 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Warehouse Name', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Racks  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_3 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig3 = plt.Figure(marks=[bar_chart_3, bar_labels_3], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig3.layout.width = 'auto'
fig3.layout.height = 'auto'
fig3.layout.min_height = '300px'


def fetch_profit_data(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/most/rack"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_profit(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_rack_data = fetch_profit_data(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_rack_data, dict) and "message" in new_rack_data:
            # Update the figure with zero values and the message
            bar_chart_3.set_trait('x', [])
            bar_chart_3.set_trait('y', [])
            bar_labels_3.set_trait('x', [])
            bar_labels_3.set_trait('y', [])
            bar_labels_3.set_trait('text', [])
            fig3.title = new_rack_data["message"]
        else:
            # Proceed as normal if data is present
            warehouse_names = [d['WarehouseName'] for d in new_rack_data]
            rack_counts = [d['Count of Racks per Warehouse'] for d in new_rack_data]

            # Update bar_chart and bar_labels with actual data
            bar_chart_3.set_trait('x', warehouse_names)
            bar_chart_3.set_trait('y', rack_counts)
            adjusted_rack_counts = [count + max(rack_counts) * 0.05 for count in rack_counts]
            bar_labels_3.set_trait('x', warehouse_names)
            bar_labels_3.set_trait('y', adjusted_rack_counts)
            bar_labels_3.set_trait('text', [str(count) for count in rack_counts])

            # Updating the title
            fig3.title = f'Top 10 warehouses with the most racks'



# Create a vuetify dropdown widget for warehouse IDs
dropdown = v.Select(items=warehouse_id_options_profits, v_model=warehouse_ids_profits[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown.observe(on_dropdown_change_profit)

display(fig3)

# Initial data load
on_dropdown_change_profit({'new': warehouse_ids_profits[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Warehouse Name', scale=OrdinalScale()), Axis(label='Racks  ', orientation='vertical',…

## Top 5 warehouses with the most incoming transactions


In [9]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options = [{'text': str(id), 'value': id} for id in warehouse_ids]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()

bar_chart_4 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Warehouse ID', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Transactions', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_4 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig4 = plt.Figure(marks=[bar_chart_4, bar_labels_4], axes=[x_ax, y_ax], title="Top 5 racks with quantity under the 25% capacity threshold for Warehouse")
fig4.layout.width = 'auto'
fig4.layout.height = 'auto'
fig4.layout.min_height = '300px'


def fetch_rack_data(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/most/incoming"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)

# Function to update the graph based on dropdown selection
def on_dropdown_change_rack(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_transaction_data = fetch_rack_data(warehouse_id)  # fetch_rack_data should fetch the new JSON format

        # Check if the response is a message indicating no records
        if isinstance(new_transaction_data, dict) and "message" in new_transaction_data:
            # Update the figure with zero values and the message
            bar_chart_4.set_trait('x', [])
            bar_chart_4.set_trait('y', [])
            bar_labels_4.set_trait('x', [])
            bar_labels_4.set_trait('y', [])
            bar_labels_4.set_trait('text', [])
            fig4.title = new_transaction_data["message"]
        else:
            # Extracting WarehouseID and Count of Incoming Transactions
            warehouse_ids = [str(data['WarehouseID']) for data in new_transaction_data]
            transaction_counts = [data['Count of Incoming Transactions per Warehouse'] for data in new_transaction_data]

            bar_chart_4.x = warehouse_ids
            bar_chart_4.y = transaction_counts

            # Update labels
            bar_labels_4.x = warehouse_ids
            bar_labels_4.y = [count + max(transaction_counts) * 0.05 for count in transaction_counts]  # Slightly above each bar
            bar_labels_4.text = [f'{count}' for count in transaction_counts]

            fig4.title = f'Top 5 warehouses with the most incoming transactions'


# Create a vuetify dropdown widget for warehouse IDs
dropdown4 = v.Select(items=warehouse_id_options, v_model=warehouse_ids[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown4.observe(on_dropdown_change_rack)

display(fig4)

# Initial data load
on_dropdown_change_rack({'new': warehouse_ids[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Warehouse ID', scale=OrdinalScale()), Axis(label='Transactions', orientation='vertica…

## Top 5 warehouses that delivers the most exchanges

In [10]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_5 = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options_5 = [{'text': str(id), 'value': id} for id in warehouse_ids_5]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_5 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Warehouse Name', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Delivered Exchanges  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_5 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig5 = plt.Figure(marks=[bar_chart_5, bar_labels_5], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig5.layout.width = 'auto'
fig5.layout.height = 'auto'
fig5.layout.min_height = '300px'


def fetch_data_5(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/most/deliver"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_5(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_5 = fetch_data_5(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_5, dict) and "message" in new_data_5:
            bar_chart_5.set_trait('x', [])
            bar_chart_5.set_trait('y', [])
            bar_labels_5.set_trait('x', [])
            bar_labels_5.set_trait('y', [])
            bar_labels_5.set_trait('text', [])
            fig5.title = new_data_5["message"]
        else:
            # Extract Warehouse Names and Counts of Delivered Exchanges
            warehouse_names = [d["WarehouseName"] for d in new_data_5]
            delivered_exchanges_counts = [d["Count of Delivered Exchanges per Warehouse"] for d in new_data_5]

            bar_chart_5.set_trait('x', warehouse_names)
            bar_chart_5.set_trait('y', delivered_exchanges_counts)

            # Update labels
            bar_labels_5.set_trait('x', warehouse_names)
            bar_labels_5.set_trait('y', [count + max(delivered_exchanges_counts) * 0.05 for count in delivered_exchanges_counts])
            bar_labels_5.set_trait('text', [str(count) for count in delivered_exchanges_counts])

            fig5.title = f'Top 5 warehouses that delivers the most exchanges'




# Create a vuetify dropdown widget for warehouse IDs
dropdown5 = v.Select(items=warehouse_id_options_5, v_model=warehouse_ids_5[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown5.observe(on_dropdown_change_5)

display(fig5)

# Initial data load
on_dropdown_change_5({'new': warehouse_ids_5[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Warehouse Name', scale=OrdinalScale()), Axis(label='Delivered Exchanges  ', orientati…

## tab 6

In [11]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_6 = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options_6 = [{'text': str(id), 'value': id} for id in warehouse_ids_6]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_6 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Username', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Transactions per User  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_6 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig6 = plt.Figure(marks=[bar_chart_6, bar_labels_6], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig6.layout.width = 'auto'
fig6.layout.height = 'auto'
fig6.layout.min_height = '300px'


def fetch_data_6(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/most/transactions"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_6(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_6 = fetch_data_6(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_6, dict) and "message" in new_data_6:
            bar_chart_6.set_trait('x', [])
            bar_chart_6.set_trait('y', [])
            bar_labels_6.set_trait('x', [])
            bar_labels_6.set_trait('y', [])
            bar_labels_6.set_trait('text', [])
            fig6.title = new_data_6["message"]
        else:
            # Extracting Username and Count of Transactions per User
            usernames = [user['Username'] for user in new_data_6]
            transaction_counts = [user['Count of Transactions per User'] for user in new_data_6]

            bar_chart_6.set_trait('x', usernames)
            bar_chart_6.set_trait('y', transaction_counts)

            # Update labels
            adjusted_counts = [count + max(transaction_counts) * 0.05 for count in transaction_counts]  # Slightly above each bar
            bar_labels_6.set_trait('x', usernames)
            bar_labels_6.set_trait('y', adjusted_counts)
            bar_labels_6.set_trait('text', [f'{count}' for count in transaction_counts])

            fig6.title = f'Top 3 users that made the most transactions'



# Create a vuetify dropdown widget for warehouse IDs
dropdown6 = v.Select(items=warehouse_id_options_6, v_model=warehouse_ids_6[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown6.observe(on_dropdown_change_6)

display(fig6)

# Initial data load
on_dropdown_change_6({'new': warehouse_ids_6[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Username', scale=OrdinalScale()), Axis(label='Transactions per User  ', orientation='…

## tab7  Top 3 warehouses’ cities with the most transactions

In [12]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_7 = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options_7 = [{'text': str(id), 'value': id} for id in warehouse_ids_7]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_7 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Cities', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Transactions  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_7 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig7 = plt.Figure(marks=[bar_chart_7, bar_labels_7], axes=[x_ax, y_ax], title='Incoming transactions for Selected Warehouse')
fig7.layout.width = 'auto'
fig7.layout.height = 'auto'
fig7.layout.min_height = '300px'


def fetch_data_7(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/most/city"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_7(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_7 = fetch_data_7(warehouse_id)  # This function should fetch the new data format

        # Check if the response is a message indicating no records
        if isinstance(new_data_7, dict) and "message" in new_data_7:
            bar_chart_7.set_trait('x', [])
            bar_chart_7.set_trait('y', [])
            bar_labels_7.set_trait('x', [])
            bar_labels_7.set_trait('y', [])
            bar_labels_7.set_trait('text', [])
            fig7.title = new_data_7["message"]
        else:
            # Extracting data from the new format
            warehouse_cities = [d["WarehouseCity"] for d in new_data_7]
            transaction_counts = [d["Count of Transactions per City"] for d in new_data_7]

            bar_chart_7.set_trait('x', warehouse_cities)
            bar_chart_7.set_trait('y', transaction_counts)

            # Update labels
            adjusted_transaction_counts = [count + max(transaction_counts) * 0.05 for count in transaction_counts]
            bar_labels_7.set_trait('x', warehouse_cities)
            bar_labels_7.set_trait('y', adjusted_transaction_counts)
            bar_labels_7.set_trait('text', [f'{count}' for count in transaction_counts])

            fig7.title = f'Top 3 warehouses’ cities with the most transactions'

# Create a vuetify dropdown widget for warehouse IDs
dropdown7 = v.Select(items=warehouse_id_options_7, v_model=warehouse_ids_7[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown7.observe(on_dropdown_change_7)

display(fig7)

# Initial data load
on_dropdown_change_7({'new': warehouse_ids_7[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Cities', scale=OrdinalScale()), Axis(label='Transactions  ', orientation='vertical', …

## tab 8 Top 3 warehouses with the least outgoing transactions

In [13]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_8 = [warehouse['WarehouseID'] for warehouse in warehouse_data]
warehouse_id_options_8 = [{'text': str(id), 'value': id} for id in warehouse_ids_8]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_8 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Warehouse Name', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Outgoing Transactions  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_8 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig8 = plt.Figure(marks=[bar_chart_8, bar_labels_8], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig8.layout.width = 'auto'
fig8.layout.height = 'auto'
fig8.layout.min_height = '300px'


def fetch_data_8(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/least/outgoing"
    response = requests.get(profit_url)
    profit_data = response.json()
    return (profit_data)



def on_dropdown_change_8(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_8 = fetch_data_8(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_8, dict) and "message" in new_data_8:
            bar_chart_8.set_trait('x', [])
            bar_chart_8.set_trait('y', [])
            bar_labels_8.set_trait('x', [])
            bar_labels_8.set_trait('y', [])
            bar_labels_8.set_trait('text', [])
            fig8.title = new_data_8["message"]
        else:
            # Assuming new_data_8 is a list of dictionaries as per your provided JSON
            warehouse_names = [d['WarehouseName'] for d in new_data_8]
            transaction_amounts = [d['Amount of Outgoing Transactions'] for d in new_data_8]

            bar_chart_8.set_trait('x', warehouse_names)
            bar_chart_8.set_trait('y', transaction_amounts)

            # Update labels
            bar_labels_8.set_trait('x', warehouse_names)
            bar_labels_8.set_trait('y', [amount + max(transaction_amounts) * 0.05 for amount in transaction_amounts])
            bar_labels_8.set_trait('text', [str(amount) for amount in transaction_amounts])

            fig8.title = f'Top 3 warehouses with the least outgoing transactions'



# Create a vuetify dropdown widget for warehouse IDs
dropdown8 = v.Select(items=warehouse_id_options_8, v_model=warehouse_ids_8[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown8.observe(on_dropdown_change_8)

display(fig8)

# Initial data load
on_dropdown_change_8({'new': warehouse_ids_8[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Warehouse Name', scale=OrdinalScale()), Axis(label='Outgoing Transactions  ', orienta…

# Set up voila vuetify layout
The voila vuetify template does not render output from the notebook, it only shows widget with the mount_id metadata.

In [14]:
v.Layout(pa_4=True, _metadata={'mount_id': 'content-nav'}, column=True, children=[]);

In [15]:

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[

    v.Tab(children=['warehouses most racks']),
    v.Tab(children=['warehouses most incoming transactions']),
    v.Tab(children=['warehouses most exchanges']),
    v.Tab(children=['users most transactions']),
    v.Tab(children=['cities with the most transactions']),
    v.Tab(children=['warehouses least outgoing transactions']),

    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg12=True, xl4=True, children=[
                fig3
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig4
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig5
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig6
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig7
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig8
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                
            ])

        ])
    ])
])

Tabs(children=[Tab(children=['warehouses most racks'], layout=None), Tab(children=['warehouses most incoming t…